# Function Calling Evolution: ReAct → Native → MCP

From prompting patterns to decoupled tool servers.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv('.env')

True

## 1. ReAct Pattern - Early tool use via prompting
LLMs generate: Thought → Action → (Runtime provides Observation) → Repeat

In [2]:
import os
import re
import yaml
from datetime import datetime
from dotenv import load_dotenv
from openai import AzureOpenAI
from lib.search_functions import search_microsoft_learn, url_to_markdown

load_dotenv('.env')

client = AzureOpenAI(
    azure_endpoint=os.getenv('ENDPOINT'),
    api_key=os.getenv('API_KEY'),
    api_version="2024-02-01"
)

def extract_action(text):
    match = re.search(r'Action:\s*(\w+)\((.*?)\)', text)
    return (match.group(1), match.group(2).strip().strip('"\'')) if match else (None, None)

def call_function(name, param):
    return search_microsoft_learn(param) if name == 'search_microsoft_learn' else url_to_markdown(param)

def react_chat(query):
    with open('system_prompt.prompt.yml', 'r') as f:
        system_content = yaml.safe_load(f)['messages'][0]['content']
    
    messages = [{"role": "system", "content": system_content}, {"role": "user", "content": query}]
    
    for step in range(5):
        print(f"\n🤖 AI STEP {step + 1} [{datetime.now().strftime('%H:%M:%S')}]")
        print("─" * 60)
        
        text = client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=messages,
            temperature=0,
            max_tokens=800
        ).choices[0].message.content
        
        print(text)
        
        if "Final Answer:" in text:
            print(f"\n✅ COMPLETED [{datetime.now().strftime('%H:%M:%S')}]")
            return text.split("Final Answer:")[-1].strip()
        
        func_name, param = extract_action(text)
        if func_name:
            result = call_function(func_name, param)
            observation = f"Observation: {result}"
            print(observation[:250] + ("..." if len(observation) > 250 else ""))
            messages.extend([{"role": "assistant", "content": text}, {"role": "user", "content": observation}])
        else:
            return text

if __name__ == "__main__":
    query = "Give me information about the Azure Well-Architected Framework workloads"
    start_time = datetime.now().strftime("%H:%M:%S")
    print(f"🚀 STARTING REACT AGENT [{start_time}]\nQuery: {query}\n{'=' * 80}")
    
    answer = react_chat(query)
    
    print(f"{'=' * 80}\n🎯 FINAL RESULT [{datetime.now().strftime('%H:%M:%S')}]\n{answer}")

🚀 STARTING REACT AGENT [16:56:13]
Query: Give me information about the Azure Well-Architected Framework workloads

🤖 AI STEP 1 [16:56:13]
────────────────────────────────────────────────────────────
Thought: I should search Microsoft Learn for information about the Azure Well-Architected Framework workloads.
Action: search_microsoft_learn("Azure Well-Architected Framework workloads")
Searching for:  Azure Well-Architected Framework workloads
Observation: ['https://learn.microsoft.com/en-us/azure/architecture/', 'https://learn.microsoft.com/en-us/azure/well-architected/', 'https://learn.microsoft.com/en-us/azure/cloud-adoption-framework/', 'https://learn.microsoft.com/en-us/azure/well-arc...

🤖 AI STEP 2 [16:56:15]
────────────────────────────────────────────────────────────
Thought: The most relevant links are the main Azure Well-Architected Framework page and the service guides. I will convert these URLs to markdown to provide detailed information.
Action: url_to_markdown("https://lea

## 2. Native Function Calling - Built into modern LLMs
No string parsing needed. Model directly returns structured function calls.

In [3]:
"""Minimal function call demo."""

import os
from openai import AzureOpenAI
from dotenv import load_dotenv


def demonstrate_scenario(client: AzureOpenAI, deployment: str, scenario_name: str, request_data: dict) -> None:
    """Demonstrate a specific scenario by calling Azure OpenAI and displaying the response."""
    print(f"=== {scenario_name} ===")
    
    user_message = next(msg for msg in request_data["messages"] if msg["role"] == "user")
    print(user_message["content"])
    
    response = client.chat.completions.create(
        model=deployment,
        **request_data
    )
    
    message = response.choices[0].message
    
    if hasattr(message, 'function_call') and message.function_call:
        print(f"Function: {message.function_call}")
    else:
        print(f"Response: {message.content}")
    
    print()


def main() -> None:
    """Main function to run the demo scenarios."""
    load_dotenv('.env')
    
    client = AzureOpenAI(
        azure_endpoint=os.getenv('ENDPOINT'),
        api_key=os.getenv('API_KEY'),
        api_version="2024-02-01"
    )
    
    deployment = os.getenv('DEPLOYMENT_NAME')
    
    # Scenario 1: Simple greeting
    demonstrate_scenario(
        client, deployment,
        "Scenario 1: Greeting",
        {
            "messages": [
                {
                    "role": "user",
                    "content": "Hello! Please greet me briefly."
                }
            ]
        }
    )
    
    # Scenario 2: Function call with search
    demonstrate_scenario(
        client, deployment,
        "Scenario 2: Function Call",
        {
            "messages": [
                {
                    "role": "user",
                    "content": "Search Microsoft Learn for 'Azure Well-Architected Framework workloads'"
                }
            ],
            "functions": [
                {
                    "name": "search_microsoft_learn",
                    "description": "Search Microsoft Learn for an Azure topic.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "Search phrase"
                            }
                        },
                        "required": ["query"]
                    }
                }
            ],
            "function_call": "auto"
        }
    )

if __name__ == '__main__':
    main()

=== Scenario 1: Greeting ===
Hello! Please greet me briefly.
Response: Hello! Great to see you here.

=== Scenario 2: Function Call ===
Search Microsoft Learn for 'Azure Well-Architected Framework workloads'
Function: FunctionCall(arguments='{"query":"Azure Well-Architected Framework workloads"}', name='search_microsoft_learn')



## 3. Azure Agent with Local Functions
Functions imported locally - tight coupling between agent and tools.

In [4]:
"""Azure AI Foundry Agent Service demo with function calling."""

import os
import time
import json
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool
from lib.search_functions import search_microsoft_learn, url_to_markdown

load_dotenv('.env')

def main():
    project_client = AIProjectClient(endpoint=os.environ.get("PROJECT_ENDPOINT"), credential=DefaultAzureCredential())
    
    agent = project_client.agents.create_agent(
        model=os.environ.get("DEPLOYMENT_NAME"),
        name="azure-agent",
        instructions="Search Microsoft Learn for Azure topics when asked, then convert URLs to markdown content.",
        tools=FunctionTool(functions={search_microsoft_learn, url_to_markdown}).definitions,
    )
    
    thread = project_client.agents.threads.create()
    project_client.agents.messages.create(thread_id=thread.id, role="user", content="Give me information about the Azure Well-Architected Framework workloads")
    
    run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        
        if run.status == "requires_action":
            tool_outputs = []
            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                args = json.loads(tool_call.function.arguments)
                output = search_microsoft_learn(args["query"]) if tool_call.function.name == "search_microsoft_learn" else url_to_markdown(args["url"])
                tool_outputs.append({"tool_call_id": tool_call.id, "output": json.dumps(output)})
            project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)
    
    for message in reversed(list(project_client.agents.messages.list(thread_id=thread.id))):
        if message.role == "assistant":
            for content in message.content:
                print(content.text.value)
    
    project_client.agents.delete_agent(agent.id)

if __name__ == "__main__":
    main()

Searching for:  Azure Well-Architected Framework workloads
Here is a summary of the information about the Azure Well-Architected Framework workloads:

---

# Azure Well-Architected Framework

The Azure Well-Architected Framework helps solution architects build and review reliable, secure, and high-performing workloads on Azure by providing guidance and tools based on five key pillars:

## Pillars
- **Reliability:** Design workloads for uptime and strong recovery by building redundancy and resiliency at scale.
- **Security:** Protect workloads from attacks while maintaining confidentiality and data integrity.
- **Cost Optimization:** Manage and optimize spending at organizational, architectural, and tactical levels to stay within budget.
- **Operational Excellence:** Build holistic observability, automate deployments, and reduce production issues.
- **Performance Efficiency:** Adapt quickly to changes in workload demands using best practices like horizontal scaling and robust testing.



## 4. MCP - Model Context Protocol
Tools on remote servers. Decoupled architecture. No local function imports.

In [5]:
"""Azure AI Foundry Agent with Microsoft Learn MCP Server."""

import os
import time
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval

load_dotenv('.env')

def main():
    project_client = AIProjectClient(endpoint=os.environ.get("PROJECT_ENDPOINT"), credential=DefaultAzureCredential())
    
    mcp_tool = McpTool(server_label="microsoft_docs", server_url="https://learn.microsoft.com/api/mcp")
    
    agent = project_client.agents.create_agent(
        model=os.environ.get("DEPLOYMENT_NAME"),
        name="mcp-agent",
        instructions="Use the Microsoft Learn MCP server to search for Azure documentation",
        tools=mcp_tool.definitions,
    )
    
    thread = project_client.agents.threads.create()
    project_client.agents.messages.create(thread_id=thread.id, role="user", content="Give me information about the Azure Well-Architected Framework workloads")
    
    run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            print ("Calling MCP Tool with name: ", run.required_action)

            tool_approvals = [ToolApproval(tool_call_id=tc.id, approve=True, headers=mcp_tool.headers) 
                            for tc in run.required_action.submit_tool_approval.tool_calls 
                            if isinstance(tc, RequiredMcpToolCall)]
            if tool_approvals:
                project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals)
    
    for message in reversed(list(project_client.agents.messages.list(thread_id=thread.id))):
        if message.role == "assistant" and message.text_messages:
            for text_msg in message.text_messages:
                print(text_msg.text.value)
    
    project_client.agents.delete_agent(agent.id)

if __name__ == "__main__":
    main()

Calling MCP Tool with name:  {'type': 'submit_tool_approval', 'submit_tool_approval': {'tool_calls': [{'id': 'call_EDalKzevRA09RFRYuqMDDl7j', 'type': 'mcp', 'arguments': '{"query":"Azure Well-Architected Framework workloads"}', 'name': 'microsoft_docs_search', 'server_label': 'microsoft_docs'}]}}
Here's a summary of Azure Well-Architected Framework workloads:

### What is an Azure Well-Architected Framework workload?
- In Azure's Well-Architected Framework, a **workload** is a collection of application resources, custom code, AI models, data, and supporting infrastructure that function together to deliver a specific business outcome.
- Architects break down workloads into logical components for design and optimization, analyzing interactions, data flows, and operational processes.
- Workload classes can be based on:
  - Utility and usage (e.g., web apps, batch processing, analytics)
  - Technology platform or industry alignment
  - Target audience (e.g., enterprise LOB apps, ISV soluti

## Summary

**Evolution:** ReAct (string parsing) → Native functions (built-in) → MCP (decoupled servers)

**MCP Benefits:**
- Tools and agents have independent lifecycles
- One tool server serves multiple agents
- No code duplication
- Update tools without touching agents